In [3]:
import chemprop
import pandas as pd
import numpy as np

In [49]:
original = pd.read_csv(r'./data/train_dm300_full.csv')


In [41]:
solvents = ["MeOH", "EtOH", "IPA", "Tol", "Hept", "Acet", "MeCN", "MTBE", "MeTHF", "DMF", "EtOAc"]
solvent_names = {"Methanol":"Methanol", 
                "Ethanol":"Ethanol", 
                "2-Propanol":"2-Propanol", 
                "Toluene":"Toluene", 
                "Heptane":"Heptane", 
                "Acetone":"Acetone", 
                "Acetonitrile":"Acetonitrile", 
                "Methyl tert butyl ether":"Methyl\ntert-butyl ether", 
                "2-Methly tetrahydrofurane":"2-Methyl\ntetrahydrofurane",
                "Dimethyl formamide":"Dimethyl\nformamide", 
                "Etyl acetate":"Ethyl\nacetate" }

solvent_dict = {}
for solvent, name in zip(solvents, solvent_names.keys()):
    solvent_dict[solvent] = original[original['solvent_name']==name]

In [51]:
original.to_csv(r'./data/_full.csv')

In [84]:
#original = pd.read_csv(r'C:\Users\ignaczg\Dropbox\Gergo Ignacz\Projects\AI\pims\data\perm_data_noned.csv')

for i in range(1):
    print(f'{i}th cycle')
    arguments = [
        '--data_path', './data/FINAL_LIST.csv',
        '--dataset_type', 'regression',
        '--save_dir', './data/train_results/removal/',
        '--metric', 'rmse',
        '--quiet',
        '--split_sizes', '0.795', '0.2', '0.005',
        '--target_columns', 'dm300',
        '--smiles_column', 'full_smiles',
        '--epochs', '300'
        ]


    args = chemprop.args.TrainArgs().parse_args(arguments)
    mean_score, std_score = chemprop.train.cross_validate(args=args, train_func=chemprop.train.run_training)

    arguments = [
        '--test_path',  './data/FINAL_LIST.csv',
        '--preds_path', './data/predictions/_temp_FINAL_LIST.csv',
        '--checkpoint_dir', './data/train_results/removal/',
        '--smiles_column', 'full_smiles'
        ]

    args = chemprop.args.PredictArgs().parse_args(arguments)
    preds = chemprop.train.make_predictions(args=args)

    original = pd.read_csv(r'./data/FINAL_LIST.csv')
    print('#############################')
    print('ORIGINAL',len(original))
    temp = pd.read_csv(r'./data/predictions/_temp_FINAL_LIST.csv')
    print('#############################')
    print('TEMP',len(original))

    temp.rename(columns={'dm300':'dm300_pred'}, inplace=True)

    # original['dm300_err'] = np.divide(np.subtract(temp['dm300_pred'], original['dm300']), original['dm300'])
    original['dm300_err'] = np.abs(np.subtract(temp['dm300_pred'], original['dm300']))

    # original['global_err'] = np.abs(np.divide(np.add(original['dm300_err'], np.add(original['d'], np.add(original['co2_err'], np.add(original['o2_err'], np.add(original['n2_err'], original['ch4_err']))))), 6))
    original = original.sort_values('dm300_err', ascending=False)
    original['dm300_measured'] = temp['dm300_pred']
    #original = original[5:]

    original.to_csv(r'./data/_FINAL_LIST.csv', sep=',')


0th cycle


1087it [00:00, 89069.66it/s]
100%|████████████████████████████████████| 1087/1087 [00:00<00:00, 95265.34it/s]
Fold 0
 89%|██████████████████████████████████████▏    | 16/18 [00:00<00:00, 30.38it/s]
                                                                                
 89%|██████████████████████████████████████▏    | 16/18 [00:00<00:00, 29.50it/s]
                                                                                
 89%|██████████████████████████████████████▏    | 16/18 [00:00<00:00, 29.82it/s]
                                                                                
 83%|███████████████████████████████████▊       | 15/18 [00:00<00:00, 28.82it/s]
                                                                                
 83%|███████████████████████████████████▊       | 15/18 [00:00<00:00, 28.37it/s]
                                                                                
 83%|███████████████████████████████████▊       | 15/18 [00:00<00:00, 28.

                                                                                
 94%|████████████████████████████████████████▌  | 17/18 [00:00<00:00, 29.02it/s]
                                                                                
 83%|███████████████████████████████████▊       | 15/18 [00:00<00:00, 28.04it/s]
                                                                                
 89%|██████████████████████████████████████▏    | 16/18 [00:00<00:00, 31.00it/s]
                                                                                
 89%|██████████████████████████████████████▏    | 16/18 [00:00<00:00, 29.92it/s]
                                                                                
 83%|███████████████████████████████████▊       | 15/18 [00:00<00:00, 29.36it/s]
                                                                                
 89%|██████████████████████████████████████▏    | 16/18 [00:00<00:00, 29.48it/s]
                            

 89%|██████████████████████████████████████▏    | 16/18 [00:00<00:00, 29.99it/s]
                                                                                
 89%|██████████████████████████████████████▏    | 16/18 [00:00<00:00, 29.42it/s]
                                                                                
 89%|██████████████████████████████████████▏    | 16/18 [00:00<00:00, 26.87it/s]
                                                                                
 83%|███████████████████████████████████▊       | 15/18 [00:00<00:00, 30.21it/s]
                                                                                
 89%|██████████████████████████████████████▏    | 16/18 [00:00<00:00, 31.52it/s]
                                                                                
 89%|██████████████████████████████████████▏    | 16/18 [00:00<00:00, 30.64it/s]
                                                                                
 89%|███████████████████████

 94%|████████████████████████████████████████▌  | 17/18 [00:00<00:00, 27.89it/s]
                                                                                
 83%|███████████████████████████████████▊       | 15/18 [00:00<00:00, 27.88it/s]
                                                                                
 89%|██████████████████████████████████████▏    | 16/18 [00:00<00:00, 27.83it/s]
                                                                                
 89%|██████████████████████████████████████▏    | 16/18 [00:00<00:00, 29.01it/s]
                                                                                
 83%|███████████████████████████████████▊       | 15/18 [00:00<00:00, 28.16it/s]
                                                                                
 94%|████████████████████████████████████████▌  | 17/18 [00:00<00:00, 28.04it/s]
                                                                                
 83%|███████████████████████

...

Validating SMILES
Test size = 1,087
Predicting with an ensemble of 1 models


  0%|                                                     | 0/1 [00:00<?, ?it/s]

Loading pretrained parameter "encoder.encoder.0.cached_zero_vector".
Loading pretrained parameter "encoder.encoder.0.W_i.weight".
Loading pretrained parameter "encoder.encoder.0.W_h.weight".
Loading pretrained parameter "encoder.encoder.0.W_o.weight".
Loading pretrained parameter "encoder.encoder.0.W_o.bias".
Loading pretrained parameter "ffn.1.weight".
Loading pretrained parameter "ffn.1.bias".
Loading pretrained parameter "ffn.4.weight".
Loading pretrained parameter "ffn.4.bias".
Moving model to cuda



100%|█████████████████████████████████████████████| 1/1 [00:00<00:00,  1.44it/s]

Saving predictions to ./data/predictions/_temp_FINAL_LIST.csv
Elapsed time = 0:00:01
#############################
ORIGINAL 1087
#############################
TEMP 1087


In [8]:
arguments = [
    '--test_path',  './data/_train_dm300_full.csv',
    '--preds_path', './data/predictions/_temp.csv',
    '--checkpoint_dir', './data/train_results/removal/',
    '--smiles_column', 'full_smiles'
    ]
args = chemprop.args.PredictArgs().parse_args(arguments)
preds = chemprop.train.make_predictions(args=args)


Loading training args
Setting molecule featurization parameters to default.
Loading data


867it [00:00, 84730.45it/s]
100%|█████████████████████████████████████| 867/867 [00:00<00:00, 146466.15it/s]


Validating SMILES
Test size = 867
Predicting with an ensemble of 1 models


  0%|                                                     | 0/1 [00:00<?, ?it/s]

Loading pretrained parameter "encoder.encoder.0.cached_zero_vector".
Loading pretrained parameter "encoder.encoder.0.W_i.weight".
Loading pretrained parameter "encoder.encoder.0.W_h.weight".
Loading pretrained parameter "encoder.encoder.0.W_o.weight".
Loading pretrained parameter "encoder.encoder.0.W_o.bias".
Loading pretrained parameter "ffn.1.weight".
Loading pretrained parameter "ffn.1.bias".
Loading pretrained parameter "ffn.4.weight".
Loading pretrained parameter "ffn.4.bias".
Moving model to cuda



100%|█████████████████████████████████████████████| 1/1 [00:00<00:00,  1.29it/s]

Saving predictions to ./data/predictions/_temp.csv
Elapsed time = 0:00:01


In [54]:
mecn = pd.read_csv(r'./data/predictions/menc_final.csv')
dmf = pd.read_csv(r'./data/predictions/dmf_final.csv')
meoh = pd.read_csv(r'./data/predictions/meoh_final.csv')

In [56]:
total = pd.concat([mecn, dmf, meoh])

In [74]:
temp = original

In [77]:
temp = temp[temp['solvent_name']!='Dimethyl formamide']

In [79]:
full_list = pd.concat([temp, total])

In [82]:
len(full_list)

1087

In [83]:
full_list.to_csv(r'./data/FINAL_LIST.csv')